In [1]:
import nltk
import numpy as np 

from HMM import unsupervised_HMM
from HMM_helper import (
    text_to_wordcloud,
    states_to_wordclouds,
    parse_observations,
    sample_sentence,
    visualize_sparsities,
    animate_emission
)

In [2]:
raw = open("data/shakespeare.txt").read()

In [3]:
word_dict = []
syllables = {}

# Wrangle syllable data
with open('data/Syllable_dictionary.txt') as file:
    for line in file:
        word = line.strip('\n').split(' ')
        word_dict.append(word[0])
        for syl in word[1:]:
            if syl not in syllables:
                syllables[syl] = [word_dict[-1]]
            else:
                syllables[syl].append(word_dict[-1])
file.close()

word_dict = ["\n"] + word_dict

In [4]:
sonnets = []

# Wrangle sonnets using syllables
with open('data/shakespeare.txt') as file:
    for line in file:
        son = []
        if line == '\n':
            continue
        if line.strip().isdigit():
            continue
        words = line.strip().split(' ')
        
        # Add word to list
        for word in words:
            word = word.lower()
            
            # If it's a word in our word_dict
            if word in word_dict:
                son.append(word_dict.index(word))
            
            # Else check if we can strip off the punct
            elif word[1:] in word_dict:
                if word[0] not in word_dict:
                    word_dict.append(word[0])
                    syllables['0'].append(word[0])
                
                # Add punctuation and word
                son.append(word_dict.index(word[0]))
                son.append(word_dict.index(word[1:]))
                
            elif word[:-1] in word_dict:
                if word[-1] not in word_dict:
                    word_dict.append(word[-1])
                    syllables['0'].append(word[-1])
                
                # Add word and punctuation
                son.append(word_dict.index(word[:-1]))
                son.append(word_dict.index(word[-1]))
                
            continue
            
        sonnets.append(son)
file.close()

In [5]:
for key in syllables:
    syllables[key] = [word_dict.index(word) for word in syllables[key]]

In [6]:
hmm4 = unsupervised_HMM(sonnets, 2, 10, syllables)

Iteration: 10


In [7]:
token_dict = {word_dict[ind]:ind for ind in range(len(word_dict))}

In [8]:
print('Sample Sentence:\n====================')
print(sample_sentence(hmm4, token_dict, n_words=14))

Sample Sentence:
Throned yet am insufficiency with . 
 gracious in charactered and my mock the , 
 and and inheritors from decease i , 
 age and farthest thy number fell more love , 
 is is the or which draw hair be much lov'st , 
 love habit soul be taught my in my of ' 
 parts remember day breathe sweet this and find , 
 enough shape stars upon whose eyes poor whose , 
 away their waiting i wide believe it , 
 to true care thou unthrift i he that the , 
 next to and at when the and death since tongue , 
 without poor beauty express feel from to , 
 in them heart-inflaming heinous that not . 
 thy my i shows of o kind-hearted worth's , 
...
